In [4]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    # Extract the ratio parts
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    
    # Calculate risk amount based on mean candle size
    risk_amount = mean_candle_size * risk_part
    
    # Calculate reward amount based on risk reward ratio
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        # Calculate stop loss and take profit prices for buy order
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif order_type == 'sell':
        # Calculate stop loss and take profit prices for sell order
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError("order_type must be either 'buy' or 'sell'")
    
    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
mt5.initialize()
login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "USDCAD"
timeframe = mt5.TIMEFRAME_H4  # Hourly timeframe

# Define start date and end date
start_date = datetime(2022, 12, 20, 12, 0, 0, tzinfo=utc_tz)  # Specify timezone as UTC
end_date = datetime(2024, 5, 10, 12, 0, 0, tzinfo=utc_tz)  # Specify timezone as UTC

# Retrieve OHLC data from MetaTrader
ohlc_data = pd.DataFrame(mt5.copy_rates_range(symbol, timeframe, start_date, end_date))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close'] 

# Load predictions from CSV file
df_pred = pd.read_csv('predictionUSDCAD_H4Buy.csv', index_col=0)

# Ensure the index of the prediction dataframe is of datetime type
df_pred.index = pd.to_datetime(df_pred.index)

# Define a simple strategy based on predictions
class SimpleStrategy(Strategy):
    mean_candle_size = 0.0034552905636873153
    risk_reward_ratio = '2:2'  # Default value

    def init(self):
        setattr(self, 'risk_reward_ratio', self.risk_reward_ratio)  # Set the risk_reward_ratio attribute
        print(self.risk_reward_ratio)

    def next(self):
        current_time = self.data.index[-1]
        
        # Check if the current time is in the predictions index
        if current_time in df_pred.index:
            prediction = df_pred.loc[current_time, 'prediction']
            entry_price = self.data.Close[-1]
            
            if prediction == 1:
                order_type = 'buy'
                sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratio, order_type, self.mean_candle_size)
                self.buy(size=100000, sl=sl_price, tp=tp_price)  # Buy if prediction is 1

# Create and run backtest with the SimpleStrategy
bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.01)
output = bt.run()
print(output)

""" risk_reward_ratios = ['1:1', '1:2', '1:3', '1:4','2:2','2:3','2:4','2:5']

# Optimize the strategy for SL/TP ratios
stats = bt.optimize(
    risk_reward_ratio=risk_reward_ratios,
    maximize='Equity Final [$]',
    constraint=lambda p: ':' in p.risk_reward_ratio
)

# Print optimized strategy stats
print(f"Best risk_reward_ratio: {stats._strategy.risk_reward_ratio}")
print(stats)
"""
# Shutdown MT5 connection after data retrieval
mt5.shutdown()

# Plot the results of the optimized strategy
bt.plot()


2:2
Start                     2022-12-20 12:00:00
End                       2024-05-10 12:00:00
Duration                    507 days 00:00:00
Exposure Time [%]                   55.807497
Equity Final [$]                 44385.553941
Equity Peak [$]                  47211.890079
Return [%]                         343.855539
Buy & Hold Return [%]                0.120298
Return (Ann.) [%]                  183.018035
Volatility (Ann.) [%]              289.885427
Sharpe Ratio                         0.631346
Sortino Ratio                        4.582792
Calmar Ratio                         3.445209
Max. Drawdown [%]                  -53.122474
Avg. Drawdown [%]                   -7.384938
Max. Drawdown Duration      241 days 04:00:00
Avg. Drawdown Duration       12 days 18:00:00
# Trades                                  266
Win Rate [%]                        62.030075
Best Trade [%]                       0.581166
Worst Trade [%]                     -0.556091
Avg. Trade [%]                

c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Atharv\anaconda3\envs\my_env\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p3930', ...)